In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
# Load your dataset
data = pd.read_csv('/content/drive/My Drive/train1.csv')  # Update with your file path
X = data['comment_text']
y = data['identity_hate']

# Tokenize text data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
X_encoded = tokenizer(X.tolist(), padding=True, truncation=True, max_length=64, return_tensors='tf')['input_ids']

# Convert the tensor to a numpy array
X_encoded = np.array(X_encoded)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# model.trainable = True
# for layer in model.layers[:-4]:
#     layer.trainable = False

# Learning rate scheduling
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=3e-5,
#     decay_steps=10000,
#     decay_rate=0.9
# )

# Define optimizer with learning rate scheduling
# optimizer = Adam(learning_rate=lr_schedule)

# Load the pre-trained BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)  # Assuming 3 output classes

early_stopping = EarlyStopping(monitor='val_loss', patience=3)  # Stop training when validation loss stops decreasing for 3 epochs

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install transformers

In [ ]:
import transformers

In [ ]:
optimizer = Adam(learning_rate=3e-5)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=3, batch_size=16, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
8942/8942 [==============================] - 2392s 258ms/step - loss: 0.0635 - accuracy: 0.9903 - val_loss: 0.0590 - val_accuracy: 0.9894
Epoch 2/3
8942/8942 [==============================] - 2289s 256ms/step - loss: 0.0625 - accuracy: 0.9907 - val_loss: 0.0868 - val_accuracy: 0.9894
Epoch 3/3
8942/8942 [==============================] - 2290s 256ms/step - loss: 0.0622 - accuracy: 0.9907 - val_loss: 0.0670 - val_accuracy: 0.9894


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

1398/1398 [==============================] - 189s 135ms/step - loss: 0.0579 - accuracy: 0.9909
Test Loss: 0.05787649750709534
Test Accuracy: 0.9908745288848877


In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score

# Make predictions on the test set
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs.logits, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Compute F1-score
f1 = f1_score(y_test, y_pred, average='macro')
print("F1-score:", f1)

# Compute recall
recall = recall_score(y_test, y_pred, average='macro')
print("Recall:", recall)

# Compute precision
precision = precision_score(y_test, y_pred, average='macro')
print("Precision:", precision)

1398/1398 [==============================] - 201s 134ms/step
Confusion Matrix:
[[44302     0]
 [  408     0]]
F1-score: 0.49770817417876245
Recall: 0.5
Precision: 0.4954372623574145


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
threshold = 0.5  # Adjust this threshold based on your requirements

# Obtain predicted logits from the model
y_pred_probs = model.predict(X_test)
y_pred_logits = np.argmax(y_pred_probs.logits, axis=1)

# Convert predicted logits to labels based on threshold
new_labels = []

for logits in y_pred_probs.logits:
    max_score_class = np.argmax(logits)
    probs = np.exp(logits) / np.sum(np.exp(logits),  keepdims=True)
    # Split instances of classes 0 and 1 into sub-classes
    if max_score_class==0:
        if probs[0]<0.5:
            new_labels.append(0)  # Offensive a
        else:
            new_labels.append(1)  # Offensive b
    elif max_score_class==1:
        if probs[1]<0.5:
            new_labels.append(2)  # Hate speech a
        else:
            new_labels.append(3)  # Hate speech b
    else:
        new_labels.append(4)  # Neither

# Convert new labels to array
new_labels = np.array(new_labels)

1398/1398 [==============================] - 195s 140ms/step


In [ ]:
X_test

array([[  101,  1024,  2017, ...,     0,     0,     0],
       [  101,  2054,  1996, ...,     0,     0,     0],
       [  101,  4470,  3419, ...,     0,     0,     0],
       ...,
       [  101,  2330, 24540, ...,  4070,  2029,   102],
       [  101,  1000,  1024, ...,  1013, 18288,   102],
       [  101,  1000,  1996, ...,  2003,  2085,   102]], dtype=int32)

In [ ]:
np.unique(new_labels)

array([1])

In [ ]:
len(new_labels)

44710

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_test, new_labels, test_size=0.3, random_state=42)

In [ ]:
np.unique(y_train)

array([1])

In [ ]:
from transformers import TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Load the pre-trained BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)  # Assuming 5 output classes

early_stopping = EarlyStopping(monitor='val_loss', patience=3)  # Stop training when validation loss stops decreasing for 3 epochs

# Compile the model with optimizer
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model with early stopping and learning rate scheduling
model.fit(X_train, y_train, epochs=4, batch_size=16, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/4
1565/1565 [==============================] - 548s 287ms/step - loss: 0.0090 - accuracy: 0.9988 - val_loss: 6.5565e-06 - val_accuracy: 1.0000
Epoch 2/4
1565/1565 [==============================] - 413s 264ms/step - loss: 4.0393e-06 - accuracy: 1.0000 - val_loss: 1.7881e-06 - val_accuracy: 1.0000
Epoch 3/4
1565/1565 [==============================] - 423s 270ms/step - loss: 1.3756e-06 - accuracy: 1.0000 - val_loss: 7.1526e-07 - val_accuracy: 1.0000
Epoch 4/4
420/420 [==============================] - 61s 144ms/step - loss: 4.7684e-07 - accuracy: 1.0000
Test Loss: 4.7683712978141557e-07
Test Accuracy: 1.0


In [ ]:
np.unique(y_test)

array([1])

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score

# Make predictions on the test set
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs.logits, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Compute F1-score
f1 = f1_score(y_test, y_pred, average='macro')
print("F1-score:", f1)

# Compute recall
recall = recall_score(y_test, y_pred, average='macro')
print("Recall:", recall)

# Compute precision
precision = precision_score(y_test, y_pred, average='macro')
print("Precision:", precision)

420/420 [==============================] - 93s 139ms/step
Confusion Matrix:
[[13413]]
F1-score: 1.0
Recall: 1.0
Precision: 1.0


In [ ]:
np.unique(y_test)

array([1])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

conf_matrix = confusion_matrix(y_test, y_pred)

# Compute classification report (includes precision, recall, F1-score, and support)
cls_report = classification_report(y_test, y_pred)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)

# Print results
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(cls_report)
print("\nAccuracy:", accuracy)

Confusion Matrix:
[[13413]]

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     13413

    accuracy                           1.00     13413
   macro avg       1.00      1.00      1.00     13413
weighted avg       1.00      1.00      1.00     13413


Accuracy: 1.0
